In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import tdt
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import xarray as xr

import kdephys.hypno as kh
import kdephys.pd as kpd
import kdephys.xr as kx
import kdephys.utils as ku

import acr
import plotly.express as px
import acr.info_pipeline as aip

/home/kdriessen/miniconda3/envs/acrdev/lib/python3.10/site-packages/xarray/backends/cfgrib_.py:29: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [3]:
import acr.info_pipeline as aip

# Set the params to generate the subject_info file
params = {}
params["subject"] = "ACR_15"
params["raw_stores"] = ["NNXo", "NNXr", "EEGr", "EMGr"]
params["lite_stores"] = ["EEG_", "EMGr", "LFP_", "LFPo"]
params["time_stores"] = ['NNXr', 'NNXo']

params["preprocess-list"] = [
    "test-pre-snip",
    "test-post-snip",
    "pulse-calibration",
]

params["channels"] = {
    "NNXr": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
    "NNXo": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
    "EEGr": [1, 2],
    "EMGr": [1, 2],
    "EEG_": [1, 2],
    "LFP_": [2, 6, 10, 14],
    "LFPo": [2, 6, 10, 14],
}

params["stim-exps"] = {}
aip.subject_info_gen(params)

read from t=0s to t=1013.85s
Using 152.5879 Hz as SEV sampling rate for EEG_
Using 152.5879 Hz as SEV sampling rate for LFP_
Using 152.5879 Hz as SEV sampling rate for LFPo
Using 24414.0625 Hz as SEV sampling rate for NNXo
Using 1017.2526 Hz as SEV sampling rate for EEGr
Using 1017.2526 Hz as SEV sampling rate for EMGr
Using 24414.0625 Hz as SEV sampling rate for NNXr
streams for test-pre-snip are ['NRe2', 'NRe3', 'EEG_', 'LFP_', 'LFPo', 'NNXo', 'EEGr', 'EMGr', 'NNXr', 'Wavt']
using ['NNXr', 'NNXo'] to get end times for test-pre-snip
read from t=71903s to t=71933.41s
Using 24414.0625 Hz as SEV sampling rate for NNXo
Using 24414.0625 Hz as SEV sampling rate for NNXr
read from t=0s to t=1021.82s
Using 152.5879 Hz as SEV sampling rate for EEG_
Using 152.5879 Hz as SEV sampling rate for LFP_
Using 152.5879 Hz as SEV sampling rate for LFPo
Using 24414.0625 Hz as SEV sampling rate for NNXo
Using 1017.2526 Hz as SEV sampling rate for EEGr
Using 1017.2526 Hz as SEV sampling rate for EMGr
Using

In [17]:
aip.rec_time_comparitor(params["subject"])

In [4]:
hypno_file = f"/Volumes/opto_loc/Data/ACR_PROJECT_MATERIALS/acr-hypno-paths.yaml"
with open(hypno_file, "r") as f:
    hypno_info = yaml.load(f, Loader=yaml.FullLoader)

In [16]:
'ACR_15' not in hypno_info

False

In [15]:
hypno_info['ACR_15'] = {}

In [ ]:
def xcat(ds, recordings, stores=['NNXo', 'NNXr']):
    data = {}
    for store in stores:
        to_concat = []
        for recording in recordings:
            to_concat.append(ds[f'{recording}-{store}'])
        data[store] = xr.concat(to_concat, dim='datetime')
    return data

In [82]:
data = tdt.read_block('/Volumes/opto_loc/Data/ACR_9/ACR_9-laser1', evtype=['streams'], t1=0, t2=10)

read from t=0s to t=1646.77s
Using 152.5879 Hz as SEV sampling rate for EEG_
Using 610.3516 Hz as SEV sampling rate for LFP_
Using 1017.2526 Hz as SEV sampling rate for EEGr
Using 1017.2526 Hz as SEV sampling rate for EMGr
Using 24414.0625 Hz as SEV sampling rate for NNXr


In [83]:
data.streams

NRe2	[struct]
Wav2	[struct]
EEG_	[struct]
LFP_	[struct]
uS2r	[struct]
EEGr	[struct]
EMGr	[struct]
NNXr	[struct]

In [1]:
on = data.epocs.Bttn.onset
off = data.epocs.Bttn.offset

NameError: name 'data' is not defined

In [75]:
off[0] - on[0])

557.890969600001

In [76]:
t = aip.get_rec_times('ACR_12', ['laser1', 'control1'], time_stores=['NNXr'], backup_store=['EEG_'])

read from t=0s to t=1651.55s
Using 152.5879 Hz as SEV sampling rate for EEG_
Using 610.3516 Hz as SEV sampling rate for LFP_
Using 1017.2526 Hz as SEV sampling rate for EEGr
Using 1017.2526 Hz as SEV sampling rate for EMGr
Using 24414.0625 Hz as SEV sampling rate for NNXr
streams for laser1 are ['NRe2', 'Wav2', 'EEG_', 'LFP_', 'uS2r', 'EEGr', 'EMGr', 'NNXr']
using ['NNXr'] to get end times for laser1
read from t=93659s to t=93689.19s
Using 24414.0625 Hz as SEV sampling rate for NNXr
read from t=0s to t=1617.32s
Using 152.5879 Hz as SEV sampling rate for EEG_
Using 610.3516 Hz as SEV sampling rate for LFP_
Using 1017.2526 Hz as SEV sampling rate for EEGr
Using 1017.2526 Hz as SEV sampling rate for EMGr
Using 24414.0625 Hz as SEV sampling rate for NNXr
streams for control1 are ['NRe2', 'Wav2', 'EEG_', 'LFP_', 'uS2r', 'EEGr', 'EMGr', 'NNXr']
using ['NNXr'] to get end times for control1
read from t=99415s to t=99445.86s
Using 24414.0625 Hz as SEV sampling rate for NNXr


In [79]:
params = {}
params["subject"] = "ACR_9"
params["raw_stores"] = ["NNXr", "EEGr", "EMGr"]
params["lite_stores"] = ["EEG_", "EMGr", "LFP_"]
params["time_stores"] = ['NNXr']

params["preprocess-list"] = [
    "laser1-bl",
    "laser1",
    "control1-bl",
    "control1",
    "pulse-exp"
]

params["channels"] = {
    "NNXr": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
    "EEGr": [1, 2],
    "EMGr": [1, 2],
    "EEG_": [1, 2],
    "LFP_": [2, 6, 10, 14],
}

params["stim-exps"] = {"laser1": "Wav2", "pulse-exp": "Bttn"}

In [80]:
t = aip.get_rec_times('ACR_12', ['laser1', 'control1'], time_stores=params['time_stores'], backup_store=params['lite_stores'][0])

read from t=0s to t=1651.55s
Using 152.5879 Hz as SEV sampling rate for EEG_
Using 610.3516 Hz as SEV sampling rate for LFP_
Using 1017.2526 Hz as SEV sampling rate for EEGr
Using 1017.2526 Hz as SEV sampling rate for EMGr
Using 24414.0625 Hz as SEV sampling rate for NNXr
streams for laser1 are ['NRe2', 'Wav2', 'EEG_', 'LFP_', 'uS2r', 'EEGr', 'EMGr', 'NNXr']
using ['NNXr'] to get end times for laser1
read from t=93659s to t=93689.19s
Using 24414.0625 Hz as SEV sampling rate for NNXr
read from t=0s to t=1617.32s
Using 152.5879 Hz as SEV sampling rate for EEG_
Using 610.3516 Hz as SEV sampling rate for LFP_
Using 1017.2526 Hz as SEV sampling rate for EEGr
Using 1017.2526 Hz as SEV sampling rate for EMGr
Using 24414.0625 Hz as SEV sampling rate for NNXr
streams for control1 are ['NRe2', 'Wav2', 'EEG_', 'LFP_', 'uS2r', 'EEGr', 'EMGr', 'NNXr']
using ['NNXr'] to get end times for control1
read from t=99415s to t=99445.86s
Using 24414.0625 Hz as SEV sampling rate for NNXr


{'laser1': {'start': '2022-08-18T09:02:28.999999',
  'NNXr-duration': 93689.17512192,
  'NNXr-end': '2022-08-19 11:03:58.175120920'},
 'control1': {'start': '2022-08-11T09:02:15.999999',
  'NNXr-duration': 99445.77347584,
  'NNXr-end': '2022-08-12 12:39:41.773474840'}}